In [86]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from tslearn.metrics import dtw, soft_dtw
from tslearn.utils import to_time_series_dataset
from sklearn.utils.validation import _check_large_sparse
from tslearn.clustering import TimeSeriesKMeans
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn.metrics import pairwise_distances_argmin_min, jaccard_score, f1_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay

In [2]:
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: 1 if item in x else 0)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [70]:
X_df = pd.read_csv('../new_data/Persian/persian_dataset.csv', index_col=None)
Y_df = pd.read_csv('../new_data/Persian/labels.csv', usecols=['filename', 'emotions'], index_col='filename')
Y_df["emotions"] = Y_df["emotions"].apply(eval)
unique_items = to_1D(Y_df["emotions"]).unique()
labels_expanded = boolean_df(Y_df['emotions'], unique_items)
labels_expanded

,annoyed,contempt,anger,none,hatred,disgust,furious
filename,,,,,,,
persian/vid_1.mp4,1,0,0,0,0,0,0
persian/vid_10.mp4,0,1,0,0,0,0,0
persian/vid_11.mp4,0,1,0,0,0,0,0
persian/vid_12.mp4,0,1,0,0,0,0,0
persian/vid_13.mp4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
persian/vid_93.mp4,0,0,0,0,1,0,0
persian/vid_94.mp4,0,0,0,1,0,0,0
persian/vid_95.mp4,0,1,0,0,1,0,0


In [71]:
X_df['none']  = np.NaN
X_df['furious']  = np.NaN
X_df['anger']  = np.NaN
X_df['annoyed']  = np.NaN
X_df['contempt']  = np.NaN
X_df['disgust']  = np.NaN
X_df['hatred']  = np.NaN


In [5]:
X_df.head()

,filename,culture,frame,face_id,timestamp,confidence,success,AU01_r,AU02_r,AU04_r,...,pose_Rz,gaze_angle_x,gaze_angle_y,none,furious,anger,annoyed,contempt,disgust,hatred
0,persian/vid_1.mp4,persian,1,0,0.000,0.98,1,0.94,0.24,0.43,...,-0.106,0.408,0.466,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,persian/vid_1.mp4,persian,2,0,0.033,0.98,1,0.17,0.00,0.00,...,-0.113,0.478,0.490,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,persian/vid_1.mp4,persian,3,0,0.067,0.98,1,0.10,0.00,0.00,...,-0.112,0.494,0.471,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,persian/vid_1.mp4,persian,4,0,0.100,0.98,1,0.00,0.00,0.00,...,-0.112,0.540,0.466,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,persian/vid_1.mp4,persian,5,0,0.133,0.98,1,0.00,0.00,0.00,...,-0.115,0.581,0.467,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
for index, row in X_df.iterrows():
    # print(index, row)
    filename = X_df.iloc[index]['filename']
    # print(labels_expanded.loc[filename]['none':'hatred'].to_list())
    X_df.at[index,'none'] = labels_expanded.at[filename,'none']
    X_df.at[index,'furious'] = labels_expanded.at[filename,'furious']
    X_df.at[index,'anger'] = labels_expanded.at[filename,'anger']
    X_df.at[index,'annoyed'] = labels_expanded.at[filename,'annoyed']
    X_df.at[index,'contempt'] = labels_expanded.at[filename,'contempt']
    X_df.at[index,'disgust'] = labels_expanded.at[filename,'disgust']
    X_df.at[index,'hatred'] = labels_expanded.at[filename,'hatred']

### Min-Max Scaling

In [41]:
## Ablation cols
ablation_cols = ['AU01_r','AU02_r','AU04_r','AU05_r','AU06_r','AU07_r','AU09_r', 'AU10_r','AU12_r','AU14_r','AU15_r','AU17_r','AU20_r','AU23_r','AU25_r','AU26_r','AU45_r']

In [42]:
X_df = X_df.drop(columns=ablation_cols)
X_df

,filename,culture,frame,face_id,timestamp,confidence,success,pose_Rx,pose_Ry,pose_Rz,gaze_angle_x,gaze_angle_y,none,furious,anger,annoyed,contempt,disgust,hatred
0,persian/vid_1.mp4,persian,1,0,0.000,0.98,1,0.252,-0.451,-0.106,0.408,0.466,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,persian/vid_1.mp4,persian,2,0,0.033,0.98,1,0.281,-0.479,-0.113,0.478,0.490,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,persian/vid_1.mp4,persian,3,0,0.067,0.98,1,0.286,-0.511,-0.112,0.494,0.471,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,persian/vid_1.mp4,persian,4,0,0.100,0.98,1,0.286,-0.537,-0.112,0.540,0.466,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,persian/vid_1.mp4,persian,5,0,0.133,0.98,1,0.286,-0.562,-0.115,0.581,0.467,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9444,persian/vid_97.mp4,persian,35,0,1.360,0.98,1,0.100,-0.156,0.029,0.132,0.283,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9445,persian/vid_97.mp4,persian,36,0,1.400,0.98,1,0.113,-0.193,0.007,0.186,0.303,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9446,persian/vid_97.mp4,persian,37,0,1.440,0.98,1,0.127,-0.218,-0.013,0.202,0.306,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9447,persian/vid_97.mp4,persian,38,0,1.480,0.98,1,0.139,-0.229,-0.039,0.223,0.303,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [73]:
cols_to_scale = list (
    set(X_df.columns.to_list()) - set(['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success', 'none', 'furious', 'anger', 'annoyed', 'contempt', 'disgust', 'hatred'])
)
scaler = MinMaxScaler()
X_df[cols_to_scale] = scaler.fit_transform(X_df[cols_to_scale])

In [74]:
X_df.tail()

,filename,culture,frame,face_id,timestamp,confidence,success,AU01_r,AU02_r,AU04_r,...,pose_Rz,gaze_angle_x,gaze_angle_y,none,furious,anger,annoyed,contempt,disgust,hatred
9444,persian/vid_97.mp4,persian,35,0,1.36,0.98,1,0.232984,0.0,0.389262,...,0.523002,0.508136,0.484157,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9445,persian/vid_97.mp4,persian,36,0,1.40,0.98,1,0.157068,0.0,0.310962,...,0.518160,0.533240,0.496831,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9446,persian/vid_97.mp4,persian,37,0,1.44,0.98,1,0.272251,0.0,0.279642,...,0.513757,0.540679,0.498733,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9447,persian/vid_97.mp4,persian,38,0,1.48,0.98,1,0.196335,0.0,0.230425,...,0.508034,0.550442,0.496831,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9448,persian/vid_97.mp4,persian,39,0,1.52,0.98,1,0.191099,0.0,0.129754,...,0.503412,0.537889,0.494930,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Splitting into train and test

In [75]:
metadata_cols = ['frame', 'face_id', 'culture', 'filename', 'timestamp']
print(X_df.head())
videos = X_df['filename'].unique()
test_videos = pd.Series(videos).sample(frac=0.20)
train_videos = np.array(list(set(videos) - set(test_videos)))
test_df = X_df[X_df['filename'].isin(test_videos)]
metadata_test = test_df[metadata_cols]
y_test = test_df[['none', 'furious', 'anger', 'annoyed', 'contempt', 'disgust', 'hatred']].values
X_test = test_df.drop(columns = ['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success']).values

            filename  culture  frame  face_id  timestamp  confidence  success  \
0  persian/vid_1.mp4  persian      1        0      0.000        0.98        1   
1  persian/vid_1.mp4  persian      2        0      0.033        0.98        1   
2  persian/vid_1.mp4  persian      3        0      0.067        0.98        1   
3  persian/vid_1.mp4  persian      4        0      0.100        0.98        1   
4  persian/vid_1.mp4  persian      5        0      0.133        0.98        1   

     AU01_r    AU02_r    AU04_r  ...   pose_Rz  gaze_angle_x  gaze_angle_y  \
0  0.246073  0.056206  0.096197  ...  0.493286      0.636448      0.600127   
1  0.044503  0.000000  0.000000  ...  0.491746      0.668991      0.615336   
2  0.026178  0.000000  0.000000  ...  0.491966      0.676430      0.603295   
3  0.000000  0.000000  0.000000  ...  0.491966      0.697815      0.600127   
4  0.000000  0.000000  0.000000  ...  0.491305      0.716876      0.600760   

   none  furious  anger  annoyed  contempt  

In [76]:
y_test[800:805,:]


array([[0., 0., 1., 0., 1., 1., 0.],
       [0., 0., 1., 0., 1., 1., 0.],
       [0., 0., 1., 0., 1., 1., 0.],
       [0., 0., 1., 0., 1., 1., 0.],
       [0., 0., 1., 0., 1., 1., 0.]])

In [77]:
metadata_test.iloc[800:805]

,frame,face_id,culture,filename,timestamp
3720,112,0,persian,persian/vid_43.mp4,4.427
3721,113,0,persian,persian/vid_43.mp4,4.467
3722,114,0,persian,persian/vid_43.mp4,4.507
3723,115,0,persian,persian/vid_43.mp4,4.547
3724,116,0,persian,persian/vid_43.mp4,4.587


## Cross-validation

In [103]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

kfold = KFold(5, True, 1)


splits = kfold.split(train_videos)
for (i, (train, test)) in enumerate(splits):
    print(videos[train])
    print(videos[test])
    print('%d-th split: train: %d, test: %d' % (i+1, len(videos[train]), len(videos[test])))
    train_df = X_df[X_df['filename'].isin(videos[train])]
    train_metadata = train_df[metadata_cols]
    print('Training+validation data size: ', train_df.shape[0])
    y = train_df[['none', 'furious', 'anger', 'annoyed', 'contempt', 'disgust', 'hatred']].values
    X = train_df.drop(columns = ['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success']).values
    X_train, X_valid, y_train, y_valid = train_test_split(X, y)
    print('Training data size: ', X_train.shape[0])
    print('Validation data size: ', X_valid.shape[0])
    base_knn =  KNeighborsClassifier(n_neighbors=25,)
    base_lr = LogisticRegression()
    base_rf = RandomForestClassifier()
    chains = [ClassifierChain(base_knn, order='random', random_state=i)
            for i in range(7)]
    for j, model in enumerate(chains):
        model.fit(X_train, y_train)
        valid_pred = model.predict(X_valid)
        val_score =jaccard_score(y_valid, valid_pred, average='samples')
        print('Validation score in model %d: %d' % (j, val_score) )
        
    # predict on validation data
    # valid_pred_chains = np.array([chain.predict(X_valid) for chain in
    #                         chains])
    # chain_accuracy_scores = [jaccard_score(y_valid, valid_pred_chain >= .5,
    #                                 average='samples')
    #                 for valid_pred_chain in valid_pred_chains]
    
    print("Validation Jaccard Score:\n ", chain_accuracy_scores)
    # test on test data
    Y_pred_chains = np.array([chain.predict(X_test) for chain in
                            chains])
    chain_accuracy_scores = [jaccard_score(y_test, Y_pred_chain >= .5,
                                    average='samples')
                    for Y_pred_chain in Y_pred_chains]

    print("Test Jaccard Score: \n ", chain_accuracy_scores)


      



['persian/vid_1.mp4' 'persian/vid_10.mp4' 'persian/vid_12.mp4'
 'persian/vid_13.mp4' 'persian/vid_14.mp4' 'persian/vid_15.mp4'
 'persian/vid_16.mp4' 'persian/vid_17.mp4' 'persian/vid_18.mp4'
 'persian/vid_2.mp4' 'persian/vid_20.mp4' 'persian/vid_21.mp4'
 'persian/vid_22.mp4' 'persian/vid_23.mp4' 'persian/vid_24.mp4'
 'persian/vid_25.mp4' 'persian/vid_26.mp4' 'persian/vid_28.mp4'
 'persian/vid_29.mp4' 'persian/vid_3.mp4' 'persian/vid_30.mp4'
 'persian/vid_31.mp4' 'persian/vid_32.mp4' 'persian/vid_35.mp4'
 'persian/vid_36.mp4' 'persian/vid_37.mp4' 'persian/vid_38.mp4'
 'persian/vid_4.mp4' 'persian/vid_42.mp4' 'persian/vid_43.mp4'
 'persian/vid_44.mp4' 'persian/vid_45.mp4' 'persian/vid_49.mp4'
 'persian/vid_5.mp4' 'persian/vid_50.mp4' 'persian/vid_52.mp4'
 'persian/vid_53.mp4' 'persian/vid_55.mp4' 'persian/vid_56.mp4'
 'persian/vid_57.mp4' 'persian/vid_59.mp4' 'persian/vid_6.mp4'
 'persian/vid_60.mp4' 'persian/vid_63.mp4' 'persian/vid_64.mp4'
 'persian/vid_65.mp4' 'persian/vid_66.mp4' 'pe

In [102]:
from skmultilearn.adapt import MLkNN, MLTSVM
import sklearn.metrics as metrics

## MLTSVM is not compatible with later versions of numpy

kfold = KFold(5, True, 1)


splits = kfold.split(train_videos)
for (i, (train, test)) in enumerate(splits):
    # print(videos[train])
    # print(videos[test])
    print('%d-th split: train: %d, test: %d' % (i+1, len(videos[train]), len(videos[test])))
    train_df = X_df[X_df['filename'].isin(videos[train])]
    train_metadata = train_df[metadata_cols]
    print('Training+validation data size: ', train_df.shape[0])
    y = train_df[['none', 'furious', 'anger', 'annoyed', 'contempt', 'disgust', 'hatred']].values
    X = train_df.drop(columns = ['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success']).values
    X_train, X_valid, y_train, y_valid = train_test_split(X, y)
    print('Training data size: ', X_train.shape[0])
    print('Validation data size: ', X_valid.shape[0])
    classifier = MLkNN(k=8)
    # classifier = MLTSVM(c_k = 2**-1)
    prediction = classifier.fit(X_train, y_train).predict(X_valid)
    print("Validation Hamming Loss:\n ", metrics.hamming_loss(y_valid, prediction))
    y_test_pred = classifier.predict(X_test)
    print("Test Hamming Loss:\n ", metrics.hamming_loss(y_test, y_test_pred))
    

1-th split: train: 60, test: 16
Training+validation data size:  6345
Training data size:  4758
Validation data size:  1587
Validation Hamming Loss:
  0.0
Test Hamming Loss:
  0.005607315389924086
2-th split: train: 61, test: 15
Training+validation data size:  5940
Training data size:  4455
Validation data size:  1485
Validation Hamming Loss:
  0.0
Test Hamming Loss:
  0.015010351966873706
3-th split: train: 61, test: 15
Training+validation data size:  6055
Training data size:  4541
Validation data size:  1514
Validation Hamming Loss:
  0.0
Test Hamming Loss:
  0.0
4-th split: train: 61, test: 15
Training+validation data size:  6575
Training data size:  4931
Validation data size:  1644
Validation Hamming Loss:
  0.0
Test Hamming Loss:
  0.0
5-th split: train: 61, test: 15
Training+validation data size:  5985
Training data size:  4488
Validation data size:  1497
Validation Hamming Loss:
  0.0
Test Hamming Loss:
  0.015010351966873706


In [35]:
train_df.head()

,filename,culture,frame,face_id,timestamp,confidence,success,AU10_r,AU12_r,AU14_r,...,pose_Rz,gaze_angle_x,gaze_angle_y,none,furious,anger,annoyed,contempt,disgust,hatred
0,persian/vid_1.mp4,persian,1,0,0.000,0.98,1,0.98,0.66,0.0,...,-0.106,0.408,0.466,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,persian/vid_1.mp4,persian,2,0,0.033,0.98,1,1.58,0.62,0.0,...,-0.113,0.478,0.490,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,persian/vid_1.mp4,persian,3,0,0.067,0.98,1,1.55,0.58,0.0,...,-0.112,0.494,0.471,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,persian/vid_1.mp4,persian,4,0,0.100,0.98,1,1.30,0.47,0.0,...,-0.112,0.540,0.466,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,persian/vid_1.mp4,persian,5,0,0.133,0.98,1,1.37,0.75,0.0,...,-0.115,0.581,0.467,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [107]:
multilabel_confusion_matrix(y_test, Y_pred_chains[1], labels=range(0,7))
Multilabel confusion matrix puts TN at (0,0) and TP at (1,1) position thanks @Kenneth Witham for pointing out.

array([[[1474,    0],
        [   0,  182]],

       [[1296,    0],
        [   0,  360]],

       [[1223,    0],
        [   0,  433]],

       [[1396,    0],
        [   0,  260]],

       [[1092,    0],
        [  21,  543]],

       [[1412,    0],
        [   0,  244]],

       [[1348,    0],
        [ 121,  187]]])

In [108]:
from sklearn.metrics import classification_report

print(classification_report(y_test, Y_pred_chains[0], target_names=['none', 'furious', 'anger', 'annoyed', 'contempt', 'disgust', 'hatred']))

              precision    recall  f1-score   support

        none       1.00      1.00      1.00       182
     furious       1.00      1.00      1.00       360
       anger       1.00      1.00      1.00       433
     annoyed       1.00      1.00      1.00       260
    contempt       1.00      0.96      0.98       564
     disgust       1.00      1.00      1.00       244
      hatred       1.00      0.61      0.76       308

   micro avg       1.00      0.94      0.97      2351
   macro avg       1.00      0.94      0.96      2351
weighted avg       1.00      0.94      0.96      2351
 samples avg       1.00      0.96      0.97      2351



In [105]:
print(Y_pred_chains[1][800:805])

[[0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0.]]


In [100]:
print(y_test[800:805])
print(metadata_test[800:805])

[[0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0.]]
      frame  face_id  culture            filename  timestamp
3720    112        0  persian  persian/vid_43.mp4      4.427
3721    113        0  persian  persian/vid_43.mp4      4.467
3722    114        0  persian  persian/vid_43.mp4      4.507
3723    115        0  persian  persian/vid_43.mp4      4.547
3724    116        0  persian  persian/vid_43.mp4      4.587


In [24]:
print(len(y_test))
print(len(metadata_test))

1656
1656


## Break data into chunks of 50 frames or less

In [ ]:
grouped = X_df.groupby(by=['filename', 'face_id'])
## Separating test data
test_ts_list = list()
test_metadata = list()
# X_list is video/face frames, divided into 50 frames chunks
X_list = []
Y_list = []
metadata = []
frame_limit = 50
for key in grouped.groups:
    X_group = grouped.get_group(key)
    # X_group = X_group.drop(['frame', 'face_id', 'culture', 'filename', 'emotion', 'confidence','success'], axis=1)
    if len(X_group) >= frame_limit:
        splitted_group = np.array_split(X_group, math.ceil(len(X_group) / frame_limit))
        for g in splitted_group:
            X_list.append(g.drop(['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success'], axis=1).to_numpy())
            metadata.append({'filename': g.loc[g.index[0], 'filename'], 'face_id':g.loc[g.index[0], 'face_id']})
            Y_list.append(Y_df.loc[g.loc[g.index[0], 'filename']].to_list())
    else:
        X_list.append(X_group.drop(['frame', 'face_id', 'culture', 'filename', 'timestamp', 'confidence','success'], axis=1).to_numpy())
        metadata.append({'filename': X_group.loc[X_group.index[0], 'filename'],  'face_id':X_group.loc[X_group.index[0], 'face_id']})
        Y_list.append(Y_df.loc[g.loc[g.index[0], 'filename']].to_list())

In [ ]:
X_ts = to_time_series_dataset(X_list)

n_series = len(X_ts)
distance_matrix = np.zeros(shape=(n_series, n_series))

# Build distance matrix
for i in range(n_series):
    for j in range(n_series):
        x = X_ts[i]
        y = X_ts[j]
        if i != j:
            dist = soft_dtw(x, y)
            distance_matrix[i, j] = dist

https://scikit-learn.org/stable/modules/multiclass.html#classifierchain


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
X_train, X_test, Y_train, Y_test = train_test_split(X_ts, Y_list, test_size=.2,
                                                    random_state=0)

base_knn =  KNeighborsClassifier(n_neighbors=5)
chains = [ClassifierChain(base_knn, order='random', random_state=i)
          for i in range(7)]
for model in chains:
    model.fit(X_train, Y_train)

Y_pred_chains = np.array([chain.predict(X_test) for chain in
                          chains])



In [ ]:
def clean(input: str):
    input = 'persian/' + input +".mp4"
    return input
X_df = pd.read_csv('../new_data/Persian/persian_dataset.csv', index_col=None)
X_df['filename'] = X_df['filename'].apply(clean)
X_df.to_csv('../new_data/Persian/persian_dataset.csv', index=False)